In [1]:
# Parametersh5_name = '{PATH_TO_CATATAC_5_CRISPR9_ONLY_SINGLETS_H5SEURAT}'protosp_path = '{PATH_TO_PROTOSPACERS_4SET_TSV}'library = "CRISPR9"tsv_path = '{PATH_TO_TSV}'

In [2]:
library(dplyr)
library(ggplot2)
library(bedr)
library(Biostrings)
library(stringr)
library(lessR)
library(SeuratDisk)
library(Seurat)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




######################
#### bedr v1.0.7 ####
######################

checking binary availability...
  * Checking path for bedtools... PASS
    /usr/local/bin/bedtools
  * Checking path for bedops... PASS
    /usr/bin/bedops
  * Checking path for tabix... PASS
    /usr/local/bin/tabix
tests and examples will be skipped on R CMD check if binaries are missing


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, eva

In [3]:
ADD_GUIDE_A_B_COUNTS_BEFORE_DEMUX = FALSE

In [4]:
# Pass these as parameters# library      = "CRISPR7"# tsv_path <- '{PATH_TO_TSV_2}'# h5_name  <- '{PATH_TO_CATATAC_4_CRISPR7_ONLY_SINGLETS_H5SEURAT}'# protosp_path = '{PATH_TO_PROTOSPACERS_4SET_TSV_2}'

In [5]:
barcode2guide_tsv_path <- paste0(tsv_path, '{PATH_TO_LIBRARY_ALL_STRUCTURED_TSV}')xlsname <- paste0(tsv_path, '{PATH_TO_LIBRARY_PIVOT_XLSX}')tsvname <- paste0(tsv_path, '{PATH_TO_LIBRARY_PIVOT_TSV}')

In [6]:
stats <- c()

In [7]:
barcode2guide_tsv_path

[1] "/home/jupyter-ayang/notebooks/cat-atac_pipeline/cond9/output/tsv/CRISPR9_ALL_STRUCTURED.TSV"

## Load protospacer file

In [8]:
df_protosp = read.table(protosp_path, header=TRUE, comment.char="")
tfs_w_ntctl    = unique(df_protosp$X.TargetGene)
unselect_ntctl = (tfs_w_ntctl != "NTCTL") & (tfs_w_ntctl != "NTCTRL")
tfs_to_plot    = tfs_w_ntctl[unselect_ntctl]

guides_to_plot  = df_protosp$GuideName
tfs_to_plot
guides_to_plot

[1] "GATA5"   "MEOX1"   "NEUROG2"

[1] "GATA5_1"   "GATA5_2"   "MEOX1_1"   "MEOX1_2"   "NEUROG2_1" "NEUROG2_2"
[7] "NTCTL_1"   "NTCTL_2"

# Load and analyze guide capture

In [9]:
df_barcode2guide <-
    read.table(barcode2guide_tsv_path, header = TRUE, sep = "\t", quote = "")

df_barcode2guide <-
    data.frame(lapply(df_barcode2guide, trimws), stringsAsFactors = FALSE)

stats[["Guide-capture UMI counts"]] <- nrow(df_barcode2guide)

# Some ATAC barcodes are not associated with a GEX barcode
# I think those are just sequencing errors. I remove those from the list
select_valid_barcodes <- df_barcode2guide$barcode != ""
df_barcode2guide <- df_barcode2guide[select_valid_barcodes, ]

stats[["Guide-capture UMI counts after removing empty barcodes"]] <-
    nrow(df_barcode2guide)

# Append "-1" to the barcodes. Cellranger appends this number to distinguish
# different samples in aggregated files.
dummy  <- DNAStringSet(df_barcode2guide$barcode)
df_barcode2guide$barcode_dash <- paste0(dummy, "-1")

### Eliminate PCR duplicates of the UMIs from the same barcode

In [10]:
# I already deduplicated guide umis. No more dedup is necessary.
df_deduped <- df_barcode2guide

# df_deduped <- df_barcode2guide %>% distinct(gene, umi, .keep_all = TRUE)

stats[["Guide-capture UMI counts after removing PCR duplicates"]] <-
    nrow(df_deduped)

# dim(df_deduped)

In [11]:
head(df_deduped)

,atac_barcode,umi,protospacer,guide,gene,read_name,barcode,barcode_dash
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTCCGTTAGCTATTGT,CNCTTTTTTCAC,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:24867:1000,AAGCGGGTCTAAATCG,AAGCGGGTCTAAATCG-1
2,TTAGGTTTCAGCTACC,TCATTATGTATT,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:23601:1094,GCACCTAAGTCTTGAA,GCACCTAAGTCTTGAA-1
3,TAACACGCACGGTAAC,GTATACTTTGCG,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:19542:1235,TCGCATTGTCCTAATC,TCGCATTGTCCTAATC-1
4,GTCCGTTAGCTATTGT,CTCTTTTTTCAC,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:24804:1297,AAGCGGGTCTAAATCG,AAGCGGGTCTAAATCG-1
5,TTTAACCAGTTAACCT,ATCATGTATTTG,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:3360:1329,AGCTAGTTCTGTAAGC,AGCTAGTTCTGTAAGC-1
6,GTTAATCTCCTTAGTG,CAGGCGAATTAG,GCTGACCCTGCGGGGAAGAA,GATA5_1,GATA5,A01102:502:HF5LTDSX5:4:1101:23556:1329,AACCTTAAGTTCCTCA,AACCTTAAGTTCCTCA-1


# Build a df for all barcodes

Summerizing the umi counts, guide counts etc. Find the barcodes that are associated with multiple genes.

In [ ]:
# Build two new dfs to summerize guide and gene counts
df_gene_counts <- pivot(df_deduped, table, gene, barcode)
df_guide_counts <- pivot(df_deduped, table, guide, barcode)

# Drop unused columns
df_gene_counts <- select(df_gene_counts, -gene_na, -gene_n)
if(ADD_GUIDE_A_B_COUNTS_BEFORE_DEMUX){
    df_guide_counts <- select(df_guide_counts, -guide_na)
} else {
    df_barcodes <- select(df_guide_counts, -guide_na)
}

# Merge guide and gene columns
if(ADD_GUIDE_A_B_COUNTS_BEFORE_DEMUX) {
    df_barcodes <- merge(df_gene_counts, df_guide_counts, by = "barcode")
}

# n_guides is umi count
df_barcodes <- dplyr:::rename(df_barcodes, n_guides = guide_n)

# Add a "-1" to barcode names to match with multiome barcode
# names from cellranger.
dummy  <- DNAStringSet(df_barcodes$barcode)
df_barcodes$barcode_dash <- paste0(dummy, "-1")


In [ ]:
head(df_barcodes, 2)

In [ ]:
colnames(df_barcodes)

# Load multiome data

In [ ]:
seurat <- LoadH5Seurat(h5_name, verbose=FALSE)

### Intersect guide-capture barcodes with multiome barcodes

In [ ]:
# Find overlapping barcodes
multiome_barcodes       <- Cells(seurat)
df_barcodes$in_multiome <- df_barcodes$barcode_dash %in% multiome_barcodes

In [ ]:
nrow(seurat)
length(multiome_barcodes)
head(multiome_barcodes)
nrow(seurat@meta.data)
#str(seurat)
length(Cells(seurat))

In [ ]:
sample_n(df_barcodes, 5)

Write df_barcodes to TSV and XLS

In [ ]:
library(writexl)
write_xlsx(df_barcodes, xlsname)
write.table(df_barcodes, tsvname, sep = "\t", quote = FALSE, row.names = FALSE)

# Produce charts

In [ ]:
select_gt1_umi_per_cell <- df_barcodes$n_guides > 1
select_convergent       <- df_barcodes$targeted_gene_count == 1

df_barcodes_gt1         <- df_barcodes[select_gt1_umi_per_cell, ]
df_in_multiome          <- df_barcodes[df_barcodes$in_multiome, ]
df_gt1_in_multiome      <- df_barcodes_gt1[df_barcodes_gt1$in_multiome, ]
df_convergent           <- df_barcodes[select_convergent, ]

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)

ggplot(df_in_multiome, aes(x = n_guides)) +
    geom_histogram(binwidth=1) +
    labs(title = "Captured guide count", x = "#guides")+
    labs(subtitle = "multiome-overlapping guide-capture barcodes", y = "#cells")
    

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
df_dummy <- df_in_multiome[order(-df_in_multiome$n_guides),]

ggplot(df_dummy, aes(x=1:nrow(df_dummy), y = n_guides)) +
    geom_point() +
    labs(title = "Captured guide count", x = "Cell index") +
    labs(subtitle = "multiome-overlapping guide-capture barcodes", y = "#guides")

ggplot(df_dummy, aes(x=1:nrow(df_dummy), y = n_guides)) +
    geom_point() +
    labs(title = "Captured guide count", x = "Cell index") +
    labs(subtitle = "multiome-overlapping guide-capture barcodes", y = "#guides") +
    scale_y_log10()

In [ ]:
# Record stats
stats[["%overlapping"]] <-
    100 * sum(df_barcodes$in_multiome) / length(multiome_barcodes)

stats[["Guide-capture barcodes"]] <- nrow(df_barcodes)
stats[["Multiome barcodes"]]      <- length(multiome_barcodes)
stats[["Overlapping barcodes"]]   <- sum(df_barcodes$in_multiome)
stats[["Overlapping barcodes > 1 UMI"]] <- sum(df_barcodes_gt1$in_multiome)
stats[["Guide capture all convergent"]] <- nrow(df_convergent)
stats[["Guide capture overlapping convergent"]] <-
    sum(df_convergent$in_multiome)
stats[["Median guides per cell (multiome-overlapping)"]] <- median(df_in_multiome$n_guides)

# Print stats

In [ ]:
for (tag in names(stats)) {
    if (tag != "Capture efficiency") {
        text <- sprintf("%s\t%g\n", tag, stats[[tag]])
    }
    else {
        text <- sprintf("%s\t%.1f%%\n", tag, stats[[tag]])
    }
    cat(text)
}

In [ ]:
round(t(as.data.frame(stats)))